# Introduction to computational biology, MIM UW 2018-19
Notes from 1st lecture and lab, including assignments.

© Krzysztof Kowalczyk kk385830@students.mimuw.edu.pl

## Setup

### Loading the data

In [1]:
!curl http://regulomics.mimuw.edu.pl/wp/wp-content/uploads/2018/02/test_fasta.txt -o data/test_fasta.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    89  100    89    0     0     89      0  0:00:01 --:--:--  0:00:01  2870


### Imports

In [2]:
import Bio
from Bio import Seq, SeqIO
from tqdm import tqdm

from copy import deepcopy
from typing import List, Set, Dict, Tuple, NamedTuple

### Using BioPython

In [3]:
s = Seq.Seq("ATCG")
s  # object representing a sequence

Seq('ATCG')

In [4]:
s.reverse_complement()  # complementing DNA sequence

Seq('CGAT')

## Assignments

### Assignment 1

In [5]:
%%time
# we can use generator expressions and streaming for efficient data processing
seq_in = SeqIO.parse(open('data/test_fasta.txt', 'r'), format='fasta')
seq_out = (record.reverse_complement() for record in seq_in)
SeqIO.write(seq_out, 'data/test_fasta_complements.txt', format='fasta')

Wall time: 35 ms


### Assignment 2
Generate list of k-mers from a given sequence.

This will allow us to transform a known sequence of DNA into a list of k-mers,
similar to a one we might get by examining a sample using microarray.

We will later use this to test different algorithms of assembling the DNA sequence from its k-mers,
having the sequences themselves will allow us to test these algorithms.

In [6]:
s_test = Seq.Seq("CATGCAGGTCC")

In [7]:
def kmers(s: Bio.Seq.Seq, k: int, complement: bool=False):
    """ Splits a sequence into k-mers, includes complements if necessary """
    if complement:
        # not sure this is the right interpretation of the assignment
        seq = s + s.reverse_complement()
    else:
        seq = s
    for i in range(len(seq)-k+1):
        yield seq[i:i+k]

In [8]:
list(kmers(s_test, 2))

[Seq('CA'),
 Seq('AT'),
 Seq('TG'),
 Seq('GC'),
 Seq('CA'),
 Seq('AG'),
 Seq('GG'),
 Seq('GT'),
 Seq('TC'),
 Seq('CC')]

In [9]:
list(kmers(s_test, 2, complement=True))

[Seq('CA'),
 Seq('AT'),
 Seq('TG'),
 Seq('GC'),
 Seq('CA'),
 Seq('AG'),
 Seq('GG'),
 Seq('GT'),
 Seq('TC'),
 Seq('CC'),
 Seq('CG'),
 Seq('GG'),
 Seq('GA'),
 Seq('AC'),
 Seq('CC'),
 Seq('CT'),
 Seq('TG'),
 Seq('GC'),
 Seq('CA'),
 Seq('AT'),
 Seq('TG')]

### Assignment 3
Knowing what k-mers are present in a sequence, we want to determin what the sequence is.

Solving this problem is equivalent to finding an hamiltonian path in a graph, 
which consists of k-mers (nodes) an possible continuations (edges).
For example, given k-mers "AGC" and "GCT" we create an edge from "AGC" to "GCT" 
because "GCT" can be a continuation of a sequence that starts with "AGC"

Formally, we will create a directed edge from node n1 to n2, both representing k-mers, 
if (k-1)-suffix of k-mer represented by n1 is the same as (k-1)-prefix represented by the node n2.

After building this graph, the DNA sequence we want to decode is equivalent to the hamiltonian path in this graph.

In [10]:
Edge = Tuple[int, int]

class Graph(NamedTuple):
    nodes: List[Bio.Seq.Seq]
    edges: Dict[int, Set[Edge]]  # node -> set of adjacent nodes

def build_graph(kmers: List[Bio.Seq.Seq]) -> Set[Tuple[int, int]]:
    edges = dict()
    for idx1, node1 in enumerate(kmers):
        for idx2, node2 in enumerate(kmers):
            if node1[1:] == node2[:-1]:
                edge = (idx1, idx2)
                try:
                    edges[idx1].add(edge)
                except KeyError:
                    edges[idx1] = {edge}
    return Graph(kmers, edges)

In [11]:
test_3 = list(kmers(s_test, 3))
build_graph(test_3)

Graph(nodes=[Seq('CAT'), Seq('ATG'), Seq('TGC'), Seq('GCA'), Seq('CAG'), Seq('AGG'), Seq('GGT'), Seq('GTC'), Seq('TCC')], edges={0: {(0, 1)}, 1: {(1, 2)}, 2: {(2, 3)}, 3: {(3, 0), (3, 4)}, 4: {(4, 5)}, 5: {(5, 6)}, 6: {(6, 7)}, 7: {(7, 8)}})

In [12]:
def dfs(node: int, g: Graph, is_in_stack: List[bool], in_stack_count: int=1, path=[]) -> Tuple[bool, List[Edge]]:
    """
    We can use DFS algorithm to find a hamiltonian path, starting from the specified node.
    A node will be labelled "IN STACK" if it was visited, but some of its adjacent nodes were not yet visited.
    If we manage to visit all nodes, it means we must have traversed the entire hamiltonian path.
    """
    if in_stack_count == len(g.nodes):
        return True, path
    for edge in g.edges.get(node, {}):
        adjacent_node = edge[1]
        if is_in_stack[adjacent_node]:
            continue
        is_in_stack[adjacent_node] = True
        path.append(edge)
        complete, rec_path = dfs(adjacent_node, g, is_in_stack, in_stack_count+1, path)
        if complete:
            return True, rec_path
        else:
            is_in_stack[adjacent_node] = False
    return False, []

def hamiltonian(g: Graph, starting_node: int=0) -> List[Edge]:
    """ Wrapper for hamiltonian path finding DFS, starting from the specified node. """
    starting_stack = [False for _ in range(len(g.nodes))]
    starting_stack[starting_node] = True
    complete, rec_path = dfs(starting_node, g, starting_stack)
    if not complete:
        raise Exception("Graph does not have a hamiltonian path starting from the this node!")
    return rec_path

def as_sequence(g: Graph, path: List[Edge]) -> Bio.Seq.Seq:
    """ Assembles a path in a graph into a BioPython DNA sequence. """
    seq = g.nodes[path[0][0]]  # entire k-mer from starting node
    for edge in path:
        following_node = g.nodes[edge[1]]
        suffix = following_node[-1]
        seq += suffix  # append last item of each of the following nodes
    return seq

In [13]:
graph = build_graph(test_3)
path_h = hamiltonian(graph)
result = as_sequence(graph, path_h)
print(path_h, result)
assert(s_test == result)

[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8)] CATGCAGGTCC


In the implemented scenario, we assume that we know which node is the first one.
However, if that assumption cannot be made, the problem requires running DFS algorithm 
from each of the nodes in graph, as the hamiltonian path problem is NP-complete.

Instead, we can use a simple trick to find the path by constructing another graph:

Given a graph such as defined above, we can transform it in a following way:
1. Use all (k-1)-prefixes and (k-1)-suffixes of each node in a graph as nodes
2. For each such prefix and suffix, w1 and w2, create an edge w1 --> w2

In such a graph, each original k-mer is repressented by an edge between its prefix and suffix.
Therefore, finding an eulerian path in the new graph is equivalent to finding a hamiltonian path in the base graph.

In [14]:
def build_eulerian_graph(kmers: List[Bio.Seq.Seq]):
    edges = dict()
    nodes = list()
    for seq in kmers:
        n1, n2 = seq[:-1], seq[1:]
        try:
            i1 = nodes.index(n1)
        except ValueError:
            nodes.append(n1)
            i1 = len(nodes)-1
        try:
            i2 = nodes.index(n2)
        except ValueError:
            nodes.append(n2)
            i2 = len(nodes)-1
        edge = (i1, i2)
        try:
            edges[i1].add(edge)
        except KeyError:
            edges[i1] = {edge}
    return Graph(nodes, edges)

In [15]:
def eulerian(g: Graph, starting_node: int=0) -> List[Edge]:
    """ Hierholzer's algorithm for finding eulerian path from a given starting node. """
    adj = deepcopy(g.edges)  # we need a mutable copy
    edge_count = {
        i: len(adj.get(i, {})) 
        for i in range(len(g.nodes))
    }
    stack = [starting_node]
    eulerian_path = []
    current_node = starting_node
    while len(stack) > 0:
        if edge_count[current_node] == 0:
            eulerian_path.append(current_node)
            current_node = stack.pop()
        else:
            stack.append(current_node)
            next_node = adj[current_node].pop()[1]
            edge_count[current_node] -= 1
            current_node = next_node
    # transform path to desired output format
    eulerian_path.reverse()
    path_edges = [(i1, i2) for i1, i2 in zip(eulerian_path, eulerian_path[1:])]
    return path_edges

In [16]:
graph = build_eulerian_graph(test_3)
path = eulerian(graph)
result = as_sequence(graph, path)
print(path, result)
assert(s_test == result)

[(0, 1), (1, 2), (2, 3), (3, 0), (0, 4), (4, 5), (5, 6), (6, 7), (7, 8)] CATGCAGGTCC


## Home assignment

Given a collection of DNA sequences, we want to find a minimal k such that 
there exists a set of probes of length k that fits the collection of sequences.

The set of probes fits a collection of sequences, if every probe is complementary only to a single, unique sequence.

### Test data

In [17]:
!wsl wget http://regulomics.mimuw.edu.pl/wp/wp-content/uploads/2018/02/yeast.fa_1.gz -O - -o /dev/null | wsl gzip -d > data/yeast.fa


gzip: stdin: unexpected end of file


Some quick exploration to see what we're dealing with:

In [18]:
input_file = './data/yeast.fa'

In [19]:
io = SeqIO.parse(input_file, format='fasta')
record_lengths = [len(record) for record in io]
max(record_lengths), min(record_lengths), len(record_lengths)

(14733, 9, 4339)

In [20]:
for record in SeqIO.parse(input_file, format='fasta'):
    print(record)
    break

ID: YPR161C
Name: YPR161C
Description: YPR161C <unknown description>
Number of features: 0
Seq('ATGAGTGATAATGGTTCCCCCGCGGTTCTTCCAAAAACCGAATTTAATAAATAC...TAG', SingleLetterAlphabet())


### Solution

In [21]:
def is_possible_probe_length(record: Bio.SeqRecord.SeqRecord, length: int) -> bool:
    subseqs = set()
    record_len = len(record)
    for i in range(record_len // length + 1):
        subseq = str(record[i:min(i+length,record_len)])
        if subseq in subseqs:
            return False
        else:
            subseqs.add(subseq)
    return True

def find_minimal_probe_length(record: Bio.SeqRecord.SeqRecord) -> int:
    min_possible, max_possible = 2, len(record)
    while min_possible < max_possible:
        current_length = (max_possible + min_possible) // 2
        if is_possible_probe_length(record, current_length):
            max_possible = current_length
        else:
            min_possible = current_length+1
    assert(min_possible == max_possible)
    return min_possible

In [22]:
find_minimal_probe_length(record)  # quick check

8

In [25]:
def find_minima(record_generator, total=len(record_lengths)):
    for idx, record in tqdm(enumerate(record_generator), total=total):
        if idx==total:
            break
        else:
            yield (record.id, find_minimal_probe_length(record))

In [26]:
%%time
minima = dict(find_minima(SeqIO.parse('./data/yeast.fa', format='fasta'), total=len(record_lengths)//10))

 99%|███████████████████████████████████████████████████████████████████████████████▎| 429/433 [00:07<00:00, 63.85it/s]


Wall time: 7.15 s


Time check: < 10 seconds to calculate results for 1/10 of the dataset, not bad.
This means we should expect entire dataset to be calculated in about 1 minute.

In [34]:
sample_keys = list(minima.keys())[:20]
for key in sample_keys:
    print(key, minima[key])

YPR161C 8
YOL138C 9
YDR395W 9
YPL015C 7
YCL050C 7
YMR193W 7
YGR053C 7
YOR280C 7
YGR097W 10
YHR215W 8
YKL025C 8
YKR011C 7
YEL004W 7
YJL200C 10
YLL027W 8
YLR045C 9
YDR348C 9
YPL183C 9
YGR003W 10
YBL095W 7


In [35]:
%%time
all_minima = dict(find_minima(SeqIO.parse('./data/yeast.fa', format='fasta'), total=len(record_lengths)))

100%|██████████████████████████████████████████████████████████████████████████████| 4339/4339 [01:12<00:00, 59.66it/s]


Wall time: 1min 12s
